# Run REopt API located on localhost or NREL server

## Initialization

In [ ]:
%matplotlib widget
#%cd ..
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import copy
import os
from src.post_and_poll import get_api_results

In [ ]:
# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

## Load a previously saved API response .json file instead of running REopt

In [ ]:
response_json = 'response_1'
with open('saved_responses/' + response_json + '.json', 'rb') as handle:
    api_response = json.load(handle)

## Scenario Inputs (Post), if wanting to do a new API call

In [ ]:
post_1 = {
  "Scenario": {
    "timeout_seconds": 400,
    "optimality_tolerance_techs": 0.001,
    "Site": {
      "latitude": 39.168205,
      "longitude": -96.6899444,
      "LoadProfile": {
        "doe_reference_name": "Hospital"
      },       
      "ElectricTariff": {
        "net_metering_limit_kw": 0.0,
        "blended_annual_rates_us_dollars_per_kwh": 0.1,
        "blended_annual_demand_charges_us_dollars_per_kw": 5.0
      }
    }
  }
}

## Save the post to a .json file for sharing or future loading in

In [ ]:
# Convert python dictionary post into json and save to a .json file
post_save = post_1
with open("posts/" + 'post_1.json', 'w') as fp:
    json.dump(post_save, fp)

## Or load in a saved .json file for the post

In [ ]:
# Load a json into a python dictionary
load_post = "post_1"
with open("posts/" + load_post + ".json", 'r') as fp:
    post_1 = json.load(fp)

### Enter the root_url based on the "location" of the server that's hosting the API
### and your API key from developer.nrel.gov

In [ ]:
# Enter the branch name here, only used for development/TestRanch and staging servers
branch_name = "ghp-updates"

# Your API Key, only needed for production server
API_KEY = "uaz52dr5KTT5Qs5U72rS70hw3IYeHVEyAaDUFQvo"

# Local host, DO NOT use the first one if hosting API on Docker
#root_url = "http://localhost:8000/v1"  # Cannot access host localhost from "within Docker container" as this is configured
#root_url = "http://host.docker.internal:8000/v1"  # Must use this or below (gateway.docker.internal) for accessing host localhost
root_url = "http://gateway.docker.internal:8000/v1"  # This one **might** work for both linux and Windows - how about WSL2?

# Development/TestRanch server
#root_url = "https://" + branch_name + "-reopt-dev-api.its.nrel.gov/v1"

# Staging server
#root_url = "https://" + branch_name + "-reopt-stage-api.its.nrel.gov/v1"

# Production server
root_url = "https://developer.nrel.gov/api/reopt/v1"

## POST and poll (period GET request) API to get a new result, if not loading in a previous response
## ...This may take a while!

In [ ]:
api_response = get_api_results(post=post_1, 
                               API_KEY=API_KEY, 
                               api_url=root_url, 
                               results_file='outputs/my_results.json', 
                               run_id=None)

### If you get disconnected from the polling function but you think it ran, 
### copy the run_uuid from the log above to manually GET the results:

In [ ]:
run_uuid = "301c2514-82c5-41d2-a5d6-5d2decbf6a52"
results_url = root_url + '/job/' + run_uuid + '/results/?api_key=' + API_KEY
resp = requests.get(url=results_url, verify=False)
api_response = json.loads(resp.text)

In [ ]:
api_response["outputs"]["Scenario"]["run_uuid"]

## Get summary of results

In [ ]:
print("NPV ($) = ", api_response["outputs"]["Scenario"]["Site"]["Financial"]["npv_us_dollars"])
print("Capital Cost, Net ($) = ", api_response["outputs"]["Scenario"]["Site"]["Financial"]["net_capital_costs"])
tech_list = ["PV", "Wind", "Storage", "CHP", "Generator", "HotTES", "ColdTES", "AbsorptionChiller", "GHP", "NewBoiler", "SteamTurbine"]
for tech in tech_list:
    if tech in post_1["Scenario"]["Site"].keys():
        if tech == "GHP":
            print("GHX Number of Boreholes = ", api_response["outputs"]["Scenario"]["Site"][tech]["ghpghx_chosen_outputs"].get("number_of_boreholes"))
            print("GHP Heat Pump Capacity (ton) = ", api_response["outputs"]["Scenario"]["Site"][tech]["ghpghx_chosen_outputs"].get("peak_combined_heatpump_thermal_ton"))
        # PV and Storage are considered if the POST does not explicitly make max_[size] == 0
        if tech == "PV" and post_1["Scenario"]["Site"]["PV"]["max_kw"] == 0:
            pass
        elif tech == "Storage" and (post_1["Scenario"]["Site"]["Storage"]["max_kw"] == 0 or post_1["Scenario"]["Site"]["Storage"]["max_kwh"] == 0):
            pass
        else:
            for size_name_value in [(key, val) for key, val in api_response["outputs"]["Scenario"]["Site"][tech].items() if "size" in key]:
                print(tech + " " + size_name_value[0], " = ", size_name_value[1])
    elif tech in ["PV", "Storage"]:
        for size_name_value in [(key, val) for key, val in api_response["outputs"]["Scenario"]["Site"][tech].items() if "size" in key]:
                print(tech + " " + size_name_value[0], " = ", size_name_value[1])                

In [ ]:
elec_savings_year = api_response["outputs"]["Scenario"]["Site"]["ElectricTariff"]["year_one_bill_bau_us_dollars"] - \
    api_response["outputs"]["Scenario"]["Site"]["ElectricTariff"]["year_one_bill_us_dollars"]
fuel_savings_year = api_response["outputs"]["Scenario"]["Site"]["FuelTariff"]["year_one_boiler_fuel_cost_bau_us_dollars"] - \
    api_response["outputs"]["Scenario"]["Site"]["FuelTariff"]["year_one_boiler_fuel_cost_us_dollars"]
savings_year = elec_savings_year + fuel_savings_year
payback = api_response["outputs"]["Scenario"]["Site"]["Financial"]["net_capital_costs"] / savings_year
payback

## Do some custom response/results data processing

## Save API response and scenario data into a .pickle file for later use

In [ ]:
response_save = api_response
file_name_to_save = "response_1"
with open("saved_responses/" + file_name_to_save + ".json", 'w') as fp:
    json.dump(response_save, fp)